# Notebook for development and testing

In [3]:
from DataLoader import *

data = ship_dataset("/home/franko/Desktop/DSelimovcProject/Dataset/", 'train')
data[0]

(array([ 0.244292,  0.062692, -0.14715 , ...,  0.104346, -0.027009,
       -0.08672 ]), array([ 0.666963,  0.205666, -0.418569, ...,  0.30228 , -0.056687,
       -0.27051 ]), array([ 2.089e-03, -5.790e-04, -1.049e-03, ...,  6.110e-04,  8.000e-05,
       -7.000e-06])) (array(8.9138), array(7.8355), array(2.5681)) /home/franko/Desktop/DSelimovcProject/Dataset/train/IO_2694.json
